In [1]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Vertex AI Search

### Install adk

In [2]:
%pip install --upgrade --quiet google-adk

In [3]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[0]
print(MODEL_NAME)

gemini-2.0-flash


In [4]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [5]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION




## adk with tools

In [6]:
import asyncio

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import VertexAiSearchTool

In [7]:
VERTEXAI_SEARCH_DATASTORE_ID = "projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348"

vertex_search_tool = VertexAiSearchTool(data_store_id=VERTEXAI_SEARCH_DATASTORE_ID)

In [8]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using funtion tool when needed.

"""

agent = LlmAgent(
      name="vertexai_search_app",
      model=MODEL_NAME,
      description="Agent to answer questions using Vertex AI Search Tool.",
      instruction="I can answer your questions by searching vertex ai search took. ",
      tools=[vertex_search_tool],
)

In [9]:
from google.adk.sessions import InMemorySessionService
from google.adk import Agent, Runner

APP_NAME = "vertexai_search_app"
USER_ID = "shins"

session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID)


## Sync call

In [10]:
from google.adk.runners import Runner

runner = Runner(
    agent=agent, app_name=session.app_name, session_service=session_service
)

query = "개인정보 보호법에 대해서 알려주세요. ?"
content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    # pprint(event)

    if event.is_final_response():
        final_response = event.content.parts[0].text
        pprint(f"Agent Response: {final_response}\n")
    else:
        pprint("No results")


('Agent Response:  개인정보 보호법은 개인 정보의 수집, 이용, 제공, 관리 등에 관한 기본적인 원칙을 정하여 국민의 권익을 '
 '보호하고, 개인 정보의 오남용을 방지하는 것을 목적으로 하는 법률입니다.\n'
 '\n'
 '주요 내용은 다음과 같습니다:\n'
 '\n'
 '*   개인정보 처리 기준: 개인정보처리자는 개인정보를 수집, 이용, 제공할 때 정보 주체의 동의를 얻어야 하며, 법률에 명시된 경우 '
 '외에는 개인정보를 목적 외로 이용하거나 제3자에게 제공할 수 없습니다.\n'
 '*   정보 주체의 권리: 정보 주체는 자신의 개인정보에 대한 열람, 정정, 삭제, 처리 정지 등을 요구할 수 있습니다.\n'
 '*   개인정보 보호 책임자 지정: 개인정보처리자는 개인정보 보호에 관한 업무를 총괄하여 책임질 개인정보 보호 책임자를 지정해야 '
 '합니다.\n'
 '*   안전 조치 의무: 개인정보처리자는 개인정보가 분실, 도난, 유출, 위조, 변조 또는 훼손되지 않도록 안전하게 관리해야 합니다.\n'
 '*   개인정보 침해 시 구제: 개인정보 침해 시에는 개인정보분쟁조정위원회에 분쟁 조정을 신청하거나 법원에 소송을 제기하여 구제받을 수 '
 '있습니다.\n'
 '\n'
 '자세한 내용은 국가법령정보센터에서 "개인정보 보호법"을 검색하여 확인하실 수 있습니다.\n'
 '\n')


### Async call


In [11]:
import nest_asyncio
nest_asyncio.apply()

# Agent Interaction Function
async def call_vsearch_agent_async(query):
    print("\n--- Running Vertex AI Search Agent ---")
    print(f"Query: {query}")

    runner_vsearch = Runner(
        agent=agent, app_name=session.app_name, session_service=session_service
    )

    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "No response received."

    try:
        async for event in runner_vsearch.run_async(
            user_id=session.user_id, session_id=session.id, new_message=content
        ):
            # Like Google Search, results are often embedded in the model's response.
            if event.is_final_response() and event.content and event.content.parts:
                final_response_text = event.content.parts[0].text.strip()
                print(f"Agent Response: {final_response_text}")
                # You can inspect event.grounding_metadata for source citations
                if event.grounding_metadata:
                    print(f"  (Grounding metadata found with {len(event.grounding_metadata.grounding_attributions)} attributions)")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Ensure your datastore ID is correct and the service account has permissions.")
    print("-" * 30)

# --- Run Example ---
async def run_vsearch_example():
    # Replace with a question relevant to YOUR datastore content
    await call_vsearch_agent_async("개인정보 보호법이란?")
    await call_vsearch_agent_async("신용정보 보호법이란?")


In [12]:
try:
    asyncio.run(run_vsearch_example())
except RuntimeError as e:
    if "cannot be called from a running event loop" in str(e):
        print("Skipping execution in running event loop (like Colab/Jupyter). Run locally.")
    else:
        raise e


--- Running Vertex AI Search Agent ---
Query: 개인정보 보호법이란?
Agent Response: 개인정보 보호법은 개인 정보의 수집, 이용, 제공 및 관리에 관한 기본적인 규칙을 설정하여 사람들의 권리를 보호하고 개인 정보의 남용을 방지하기 위한 법률입니다.

주요 내용:

*   개인 정보 처리 기준: 개인 정보 처리자는 개인 정보를 수집, 사용 및 제공 할 때 데이터 주체의 동의를 얻어야 합니다. 법률에 명시된 경우를 제외하고는 개인 정보를 원래 목적 이외의 목적으로 사용하거나 제3자에게 제공 할 수 없습니다.
*   데이터 주체의 권리: 데이터 주체는 자신의 개인 정보를보고, 수정, 삭제 또는 일시 중지 할 권리가 있습니다.
*   개인 정보 보호 책임자 지정: 개인 정보 처리자는 개인 정보 보호와 관련된 업무를 감독하고 책임을 지는 개인 정보 보호 책임자를 지정해야합니다.
*   보안 조치 의무: 개인 정보 처리자는 개인 정보가 손실, 도난, 유출, 위조, 변경 또는 손상되지 않도록 안전하게 관리해야합니다.
*   개인 정보 침해에 대한 구제: 개인 정보가 침해 된 경우, 개인 정보 분쟁 조정위원회에 조정을 신청하거나 법원에 소송을 제기하여 구제를받을 수 있습니다.

자세한 내용은 국가 법령 정보 센터에서 "개인 정보 보호법"을 검색하여 확인하십시오.
An error occurred: 'GroundingMetadata' object has no attribute 'grounding_attributions'
Ensure your datastore ID is correct and the service account has permissions.
------------------------------

--- Running Vertex AI Search Agent ---
Query: 신용정보 보호법이란?
Agent Response: 신용정보 보호법은 신용정보의 이용과 보호에 관한 법률을 의미합니다. 이 법은 신용정보의 오

## End of notebook